# CS246 - Colab 5
## PageRank

### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [0]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [78]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 281903
Number of edges: 2312497
Average in degree:   8.2032
Average out degree:   8.2032


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

In [79]:
# YOUR CODE HERE
LWCC_nodes_list = max(nx.weakly_connected_components(G), key=len)
LWCC = G.subgraph(LWCC_nodes_list)
print(nx.info(LWCC))

Name: 
Type: DiGraph
Number of nodes: 255265
Number of edges: 2234572
Average in degree:   8.7539
Average out degree:   8.7539


Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [0]:
# YOUR CODE HERE
pr = nx.pagerank(LWCC)

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using for both methods ```seed = 1```, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.


In [81]:
# YOUR CODE HERE
random_graph = nx.fast_gnp_random_graph(255265, p = 0.00008, seed = 1)
pr_random_graph = nx.pagerank(random_graph)
print(nx.info(random_graph))

Name: 
Type: Graph
Number of nodes: 255265
Number of edges: 2606386
Average degree:  20.4210


In [83]:
BA_graph = nx.barabasi_albert_graph(n = 255265, m = 9, seed = 1)
pr_BA_graph = nx.pagerank(BA_graph)
print(nx.info(BA_graph))

Name: 
Type: Graph
Number of nodes: 255265
Number of edges: 2297304
Average degree:  17.9994


In [0]:
sorted_pr_random_graph = sorted(pr_random_graph.items(), key=lambda x: -x[1])
sorted_pr_BA_graph = sorted(pr_BA_graph.items(), key=lambda x: -x[1])

In [87]:
list_sorted_pr_random_graph = list(sorted_pr_random_graph)
list_sorted_pr_BA_graph = list(sorted_pr_BA_graph)
print(list_sorted_pr_random_graph[0])
print(list_sorted_pr_BA_graph[0])

(150787, 7.458986619588905e-06)
(11, 0.00042292470661201644)


Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure. 

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [0]:
# YOUR CODE HERE
list_sorted_pr_LWCC_vector = np.array(sorted(pr.values(), key = lambda x: -x)).reshape((1, -1))
list_sorted_pr_random_graph_vector = np.array(sorted(pr_random_graph.values(), key = lambda x: -x)).reshape((1, -1))
list_sorted_pr_BA_graph_vector = np.array(sorted(pr_BA_graph.values(), key = lambda x: -x)).reshape((1, -1))

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_LWCC_random = cosine_similarity(list_sorted_pr_LWCC_vector, list_sorted_pr_random_graph_vector)
cos_sim_LWCC_BA = cosine_similarity(list_sorted_pr_LWCC_vector, list_sorted_pr_BA_graph_vector)

print("Cosine similarity between the web-Stanford graph and the random generated graph: {}".format(cos_sim_LWCC_random))
print("cosine similarity between the web-Stanford graph and the Barabási-Albert graph: {}".format(cos_sim_LWCC_BA))

Cosine similarity between the web-Stanford graph and the random generated graph: [[0.10395565]]
cosine similarity between the web-Stanford graph and the Barabási-Albert graph: [[0.66124513]]


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!